In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.ensemble import RandomForestClassifier
from category_encoders import CatBoostEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Варианты возможных задач
MODEL_CONFIGS = {
        'lead_conversion': {
            'target_events': ['phone_entered', 'start_chat', 'sub_submit_success'],
        },
        'engagement': {
            'target_events': ['go_to_car_card', 'photos_all', 'search_form_search_btn'],
        },
        'upsell': {
            'target_events': ['click_pos_credit', 'click_insurance'],
        },
        'churn': {
            'target_events': ['search_form_clear'],
        }
    }

In [5]:
class TargetProcessor(BaseEstimator, TransformerMixin):
    """Трансформер для создания целевой переменной"""
    def __init__(self, target_type, target_events):
        self.target_type = target_type  # 'conversion', 'engagement', 'upsell', 'churn'
        self.target_events = target_events  # Список событий для целевого класса
        
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        df = X.copy()
        
        if self.target_type == 'churn':
            df['target'] = df['event_action'].isin(self.target_events).astype(int)
        else:
            if not df['event_action'].isin(self.target_events).any():
                raise ValueError(f"Нет целевых событий {self.target_events} в данных!")
            # Создаем Series с отметками о целевых событиях
            has_target = df['event_action'].isin(self.target_events)
            # Группируем и находим сессии с хотя бы одним целевым событием
            target_sessions = set(df.loc[has_target, 'session_id'].unique())
            # Создаем target (1 если сессия в target_sessions, иначе 0)
            df['target'] = df['session_id'].isin(target_sessions).astype(int)
        return df  

In [7]:
# Создаем целевую переменную на основе выбранной задачи
sessions = pd.read_csv('./ga_sessions.csv',low_memory=False)
hits = pd.read_csv('./ga_hits.csv',low_memory=False)

target_processor = TargetProcessor(
        target_type='lead_conversion',
        target_events=MODEL_CONFIGS['lead_conversion']['target_events']
    )
hits = target_processor.transform(hits.copy())

In [8]:
# Создаем два обучающих и два тестовых датафрейма(для имитации работы с реальными данными), предполагая, что на вход при рассчете вероятности в реальных условиях также поступит два датафрейма 
sessions_with_target = pd.merge(sessions, hits, on='session_id', how='left')
sessions_with_target['target'] = sessions_with_target['target'].fillna(0)
sesions_unique = sessions_with_target['session_id'].unique().tolist()
train_sessions, test_sessions = train_test_split(
    sesions_unique,
    test_size=0.2,
    random_state=42,
    stratify=sessions_with_target.groupby('session_id')['target'].max()  # Стратификация по target
)
train_sessions_df = sessions[sessions['session_id'].isin(train_sessions)]
train_hits_df = hits[hits['session_id'].isin(train_sessions)]
test_sessions_df = sessions[sessions['session_id'].isin(test_sessions)]
test_hits_df = hits[hits['session_id'].isin(test_sessions)]
#test_hits_df = test_hits_df.drop(columns = ['target'])
print(train_sessions_df.shape)
print(train_hits_df.shape)
print(test_sessions_df.shape)
print(test_hits_df.shape)

(1488033, 18)
(12539129, 12)
(372009, 18)
(3146090, 12)


In [9]:
# Объединяем датафреймы и заменяем (not set) и (none) на nan

class DataPreparing(BaseEstimator, TransformerMixin):
    def __init__(self, df2=None, verbose=True):
        """
        Трансформер для объединения данных с диагностическим выводом
        
        :param df2: Второй датафрейм (с target на обучении, без target на тесте)
        :param verbose: Флаг вывода диагностической информации
        """
        self.df2 = df2
        self.verbose = verbose
        
    def fit(self, X, y=None):
        """Фиктивная функция для совместимости с sklearn"""
        if self.verbose:
            print("\n=== DataPreparing.fit() ===")
            print("Входные данные (X):")
            print(f"- Индексы: {X.index[:5].tolist()}... (всего: {len(X.index)})")
            print(f"- Колонки: {X.columns.tolist()}")
            print(f"- Размер: {X.shape}")
            
            print("\nДанные df2:")
            print(f"- Индексы: {self.df2.index[:5].tolist()}... (всего: {len(self.df2.index)})")
            print(f"- Колонки: {self.df2.columns.tolist()}")
            print(f"- Размер: {self.df2.shape}")
        
        return self  # Ничего не делаем, так как структура колонок постоянная

    def transform(self, X):
        """Объединение данных с выводом диагностической информации"""
        if self.verbose:
            print("\n=== DataPreparing.transform() ===")
            print("Входные данные (X):")
            print(f"- Индексы: {X.index[:5].tolist()}... (всего: {len(X.index)})")
            print(f"- Колонки: {X.columns.tolist()}")
            print(f"- Размер: {X.shape}")
            
            print("\nДанные df2:")
            print(f"- Индексы: {self.df2.index[:5].tolist()}... (всего: {len(self.df2.index)})")
            print(f"- Колонки: {self.df2.columns.tolist()}")
            print(f"- Размер: {self.df2.shape}")
        
        # Объединяем данные
        merged = pd.merge(X, self.df2, on='session_id', how='left')
        
        # Обрабатываем специальные значения
        result = merged.replace(['(not set)', '(none)'], np.nan)
        
        if self.verbose:
            print("\nРезультат объединения:")
            print(f"- Индексы: {result.index[:5].tolist()}... (всего: {len(result.index)})")
            print(f"- Колонки: {result.columns.tolist()}")
            print(f"- Размер: {result.shape}")
            print("\nПример данных (первые 2 строки):")
            display(result.head(2))
        
        return result

In [10]:
# Создаем дополнительные фичи 
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, df, y=None):
        return self
    
    def transform(self, df):
        print("\n=== Начало transform ===")
        print(f"Исходный датафрейм: {df.shape[0]} строк, {df.shape[1]} колонок")
        print("Индексы:", df.index)
        print("Колонки:", df.columns.tolist())
        
        # Преобразование даты и времени
        df['visit_datetime'] = pd.to_datetime(df['visit_date'] + ' ' + df['visit_time'])
        df['hour'] = df['visit_datetime'].dt.hour
        df['day_of_week'] = df['visit_datetime'].dt.dayofweek
        
        print("\nПосле добавления временных признаков:")
        print(f"Размер: {df.shape[0]} строк, {df.shape[1]} колонок")
        print("Новые колонки:", [c for c in df.columns if c not in ['visit_date', 'visit_time']])
        
        # Создаем статистику по сессиям
        session_stats = (
            df.groupby('session_id')
            .agg(
                hit_count=('hit_number', 'count'),
                max_hit=('hit_number', 'max'),
                unique_actions=('event_action', 'nunique')
            )
            .reset_index()
        )
        
        print("\nСтатистика по сессиям (session_stats):")
        print(f"Размер: {session_stats.shape[0]} строк, {session_stats.shape[1]} колонок")
        print("Индексы:", session_stats.index)
        print("Колонки:", session_stats.columns.tolist())
        
        # Создаем статистику по пользователям
        user_stats = (
            df.groupby('client_id')
            .agg(
                user_sessions=('session_id', 'nunique'),
                avg_hits_per_session=('hit_number', 'mean')
            )
            .reset_index()
        )
        
        print("\nСтатистика по пользователям (user_stats):")
        print(f"Размер: {user_stats.shape[0]} строк, {user_stats.shape[1]} колонок")
        print("Индексы:", user_stats.index)
        print("Колонки:", user_stats.columns.tolist())
        
        # Объединяем все в один DataFrame
        result_df = (
            df
            .merge(session_stats, on='session_id', how='left')
            .merge(user_stats, on='client_id', how='left')
        )
        
        print("\nФинальный датафрейм (result_df):")
        print(f"Размер: {result_df.shape[0]} строк, {result_df.shape[1]} колонок")
        print("Индексы:", result_df.index)
        print("Колонки:", result_df.columns.tolist())
        
        return result_df

In [11]:
# Обрабатываем отдельно hit_page_path
class HitPagePathTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, df, y=None):
        return self
        
    def transform(self, df):
        """Преобразует путь страницы в количество компонентов"""
        print("\n=== Начало transform в HitPagePathTransformer ===")
        
        # Создаем копию во избежание предупреждений
        original_type = type(df)
        df = df.copy()
        
        print(f"Исходные данные: тип {original_type}")
        if isinstance(df, pd.DataFrame):
            print(f"Размер: {df.shape[0]} строк, {df.shape[1]} колонок")
            print("Колонки:", df.columns.tolist())
            print("Пример hit_page_path до обработки:", df['hit_page_path'].head(3).tolist())
        elif isinstance(df, np.ndarray):
            print(f"Размер массива: {df.shape}")
            print("Пример данных до обработки:", df[:3] if len(df) > 3 else df)
        
        def path_corrector(x):
            if pd.isna(x):
                return 0
            arr = str(x).split('?')
            string = " ".join(arr)
            arr2 = string.split('/')
            return len(arr2)
        
        if isinstance(df, pd.DataFrame):
            df['hit_page_path'] = df['hit_page_path'].apply(path_corrector)
            print("\nПосле преобразования hit_page_path:")
            print(f"Размер: {df.shape[0]} строк, {df.shape[1]} колонок (не изменился)")
            print("Пример hit_page_path после обработки:", df['hit_page_path'].head(3).tolist())
            print("Уникальные значения count:", df['hit_page_path'].value_counts())
        elif isinstance(df, np.ndarray):
            # Для совместимости с numpy массивами
            df = np.array([path_corrector(x) for x in df])
            print("\nПосле преобразования numpy массива:")
            print(f"Размер массива: {df.shape} (не изменился)")
            print("Пример данных после обработки:", df[:3] if len(df) > 3 else df)
        else:
            raise TypeError("Неподдерживаемый тип данных")
            
        print(f"\nВозвращаемые данные: тип {type(df)}")
        return df

In [12]:
# Один из вариантов кодирования категориальных признаков с большим количеством уникальных значений
class SimpleCatBoostEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        """
        Простой CatBoostEncoder с подробным логированием
        
        Параметры:
        cols - список колонок для кодирования
        """
        self.cols = cols
        self.encoder = None
        self._feature_names = None

    def fit(self, X, y):
        # Шаг 1: Проверка и преобразование входных данных
        print("\n=== ФАЗА ОБУЧЕНИЯ (FIT) ===")
        print("1. Проверка входных данных:")
        print(f"- Тип X: {type(X)}")
        print(f"- Форма X: {X.shape if hasattr(X, 'shape') else 'N/A'}")
        
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            if self.cols is not None:
                X.columns = self.cols
            print("\n2. Преобразование в DataFrame:")
            print("- Назначены имена колонок:", self.cols)
        
        print("\n3. Информация о DataFrame:")
        print(f"- Индексы: {X.index[:5].tolist()}... (всего: {len(X.index)})")
        print(f"- Колонки: {X.columns.tolist()}")
        print(f"- Типы данных:\n{X.dtypes}")
        print("\n4. Пример данных (первые 3 строки):")
        print(X.head(3))

        # Шаг 2: Обучение энкодера
        print("\n5. Инициализация CatBoostEncoder...")
        self.encoder = CatBoostEncoder(cols=self.cols)
        
        print("6. Обучение энкодера на указанных колонках:")
        print("- Колонки для кодирования:", self.cols)
        self.encoder.fit(X[self.cols], y)
        
        # Сохраняем имена признаков
        self._feature_names = X.columns.tolist()
        print("\n7. Обучение завершено. Сохранены имена колонок:", self._feature_names)
        
        return self

    def transform(self, X):
        # Шаг 1: Проверка входных данных
        print("\n=== ФАЗА ПРЕОБРАЗОВАНИЯ (TRANSFORM) ===")
        print("1. Проверка входных данных:")
        print(f"- Тип X: {type(X)}")
        print(f"- Форма X: {X.shape if hasattr(X, 'shape') else 'N/A'}")
        
        # Шаг 2: Преобразование данных
        if isinstance(X, pd.DataFrame):
            df = X.copy()
            print("\n2. Копирование DataFrame:")
            print("- Индексы сохранены:", df.index[:5].tolist())
        else:
            df = pd.DataFrame(X)
            if self._feature_names:
                df.columns = self._feature_names
            print("\n2. Преобразование в DataFrame:")
            print("- Назначены имена колонок:", self._feature_names)
        
        print("\n3. Информация перед кодированием:")
        print(f"- Индексы: {df.index[:5].tolist()}... (всего: {len(df.index)})")
        print(f"- Колонки: {df.columns.tolist()}")
        print("\n4. Пример данных (первые 3 строки):")
        print(df.head(3))

        # Шаг 3: Применение кодирования
        result = df.copy()
        result[self.cols] = self.encoder.transform(df[self.cols])
        
        print("\n5. Результат кодирования:")
        print(f"- Тип возвращаемых данных: {type(result)}")
        print(f"- Форма: {result.shape}")
        print(f"- Индексы: {result.index[:5].tolist()}...")
        print(f"- Новые значения колонок {self.cols}:")
        print(result[self.cols].head(3))
        
        return result

In [13]:
# Один из вариантов кодирования категориальных признаков с большим количеством уникальных значений
from sklearn.preprocessing import TargetEncoder
import pandas as pd
import numpy as np

class SimpleTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        """
        Простой TargetEncoder с подробным логированием
        
        Параметры:
        cols - список колонок для кодирования
        """
        self.cols = cols
        self.encoder = None
        self._feature_names = None

    def fit(self, X, y):
        # Шаг 1: Проверка и преобразование входных данных
        print("\n=== ФАЗА ОБУЧЕНИЯ (FIT) ===")
        print("1. Проверка входных данных:")
        print(f"- Тип X: {type(X)}")
        print(f"- Форма X: {X.shape if hasattr(X, 'shape') else 'N/A'}")
        
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            if self.cols is not None:
                X.columns = self.cols
            print("\n2. Преобразование в DataFrame:")
            print("- Назначены имена колонок:", self.cols)
        
        print("\n3. Информация о DataFrame:")
        print(f"- Индексы: {X.index[:5].tolist()}... (всего: {len(X.index)})")
        print(f"- Колонки: {X.columns.tolist()}")
        print(f"- Типы данных:\n{X.dtypes}")
        print("\n4. Пример данных (первые 3 строки):")
        print(X.head(3))

        # Шаг 2: Обучение энкодера
        print("\n5. Инициализация TargetEncoder...")
        self.encoder = TargetEncoder()
        
        print("6. Обучение энкодера на указанных колонках:")
        print("- Колонки для кодирования:", self.cols)
        self.encoder.fit(X[self.cols], y)
        
        # Сохраняем имена признаков
        self._feature_names = X.columns.tolist()
        print("\n7. Обучение завершено. Сохранены имена колонок:", self._feature_names)
        
        return self

    def transform(self, X):
        # Шаг 1: Проверка входных данных
        print("\n=== ФАЗА ПРЕОБРАЗОВАНИЯ (TRANSFORM) ===")
        print("1. Проверка входных данных:")
        print(f"- Тип X: {type(X)}")
        print(f"- Форма X: {X.shape if hasattr(X, 'shape') else 'N/A'}")
        
        # Шаг 2: Преобразование данных
        if isinstance(X, pd.DataFrame):
            df = X.copy()
            print("\n2. Копирование DataFrame:")
            print("- Индексы сохранены:", df.index[:5].tolist())
        else:
            df = pd.DataFrame(X)
            if self._feature_names:
                df.columns = self._feature_names
            print("\n2. Преобразование в DataFrame:")
            print("- Назначены имена колонок:", self._feature_names)
        
        print("\n3. Информация перед кодированием:")
        print(f"- Индексы: {df.index[:5].tolist()}... (всего: {len(df.index)})")
        print(f"- Колонки: {df.columns.tolist()}")
        print("\n4. Пример данных (первые 3 строки):")
        print(df.head(3))

        # Шаг 3: Применение кодирования
        result = df.copy()
        result[self.cols] = self.encoder.transform(df[self.cols])
        
        print("\n5. Результат кодирования:")
        print(f"- Тип возвращаемых данных: {type(result)}")
        print(f"- Форма: {result.shape}")
        print(f"- Индексы: {result.index[:5].tolist()}...")
        print(f"- Новые значения колонок {self.cols}:")
        print(result[self.cols].head(3))
        
        return result

    #def fit_transform(self, X, y):
        #"""Объединенный fit и transform"""
        #print("\n=== FIT_TRANSFORM ===")
        #self.fit(X, y)
        #return self.transform(X)

In [14]:
# Функция, которая создает pipeline и обучает моделей
def build_pipeline(df1, df2):
    print("\n=== Начало построения пайплайна ===")
    print("\n=== DataPreparing.transform() ===")
    print("Входные данные (df1):")
    print(f"- Индексы: {df1.index[:5].tolist()}... (всего: {len(df1.index)})")
    print(f"- Колонки: {df1.columns.tolist()}")
    print(f"- Размер: {df1.shape}")
    
    print("\nВходные данные (df2):")
    print(f"- Индексы: {df2.index[:5].tolist()}... (всего: {len(df2.index)})")
    print(f"- Колонки: {df2.columns.tolist()}")
    print(f"- Размер: {df2.shape}")
    
    
    # Определение фичей
    numeric_features = ['visit_number', 'hit_number', 'hour', 'day_of_week', 
                       'hit_count', 'max_hit', 'unique_actions', 
                       'user_sessions', 'avg_hits_per_session']
    other_num_features = ['hit_page_path']
    categorical_features = ['utm_source', 'utm_campaign', 'utm_adcontent','device_browser', 'geo_city']
    other_cat_features = ['device_category']
    
    print("\n=== Определены группы признаков ===")
    print(f"Числовые признаки ({len(numeric_features)}): {numeric_features}")
    print(f"Другие числовые признаки ({len(other_num_features)}): {other_num_features}")
    print(f"Категориальные признаки ({len(categorical_features)}): {categorical_features}")
    print(f"Другие категориальные признаки ({len(other_cat_features)}): {other_cat_features}")

    # Создание препроцессора
    print("\n=== Создание ColumnTransformer ===")
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', ImbPipeline([
                ('imputer', SimpleImputer(strategy='median')),
                #('scaler', StandardScaler())
            ]), numeric_features),
            ('hit_path', ImbPipeline([
                ('imputer', HitPagePathTransformer()),
                #('scaler', StandardScaler())
            ]), other_num_features),
            ('cat_cbe', ImbPipeline([
                ('categorical_imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
                ('imputer', SimpleCatBoostEncoder(cols=categorical_features)),
                #('scaler', StandardScaler())
            ]), categorical_features),
            ('cat_onehot', ImbPipeline([
                ('categorical_imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
                ('imputer', OneHotEncoder(handle_unknown='ignore')),
                #('scaler', StandardScaler())
            ]), other_cat_features)
        ],
        remainder='drop'
    )
    print("ColumnTransformer успешно создан")

    # Базовые модели
    print("\n=== Инициализация базовых моделей ===")
    base_models = [
        ('gbdt', GradientBoostingClassifier(
            random_state=42,
            n_estimators=100,
            max_depth=3
        )),
        ('xgb', XGBClassifier(
            random_state=42,
            n_estimators=100,
            learning_rate=0.1,
            max_depth=3,
            eval_metric='logloss',
            use_label_encoder=False
        )),
        ('catboost', CatBoostClassifier(
            random_state=42,
            iterations=100,
            depth=3,
            silent=True,
            thread_count=1
        ))
    ]
    
    for name, model in base_models:
        print(f"Модель {name}: {model.__class__.__name__}")

    # Мета-модель
    print("\n=== Инициализация мета-модели ===")
    meta_model = LogisticRegression(
        penalty='l2',
        C=0.1,
        random_state=42,
        max_iter=1000
    )
    print(f"Мета-модель: {meta_model.__class__.__name__}")

    # Сборка полного пайплайна
    print("\n=== Сборка полного пайплайна ===")
    pipeline = ImbPipeline([
        ('data_engineer', DataPreparing(df2=df2)),
        ('feature_engineer', FeatureEngineer()),
        ('preprocessor', preprocessor),
        ('undersampler', RandomUnderSampler(random_state=42)),
        ('stacking', StackingClassifier(
            estimators=base_models,
            final_estimator=meta_model,
            stack_method='auto',
            passthrough=False,
            n_jobs=-1
        ))
    ])
    
    print("\n=== Структура пайплайна ===")
    for i, (name, step) in enumerate(pipeline.steps):
        print(f"{i+1}. {name}: {step.__class__.__name__}")
        if name == 'stacking':
            print("   Составные модели:")
            for base_name, base_model in step.estimators:
                print(f"   - {base_name}: {base_model.__class__.__name__}")
            print(f"   Мета-модель: {step.final_estimator.__class__.__name__}")
    
    print("\n=== Пайплайн успешно построен ===")
    return pipeline

In [15]:
# Функция, которая запускает создания pipeline и обучения моделей

def train_model(df1, df2):
    """
    Обучение pipeline на объединенных данных df1 и df2
    
    Параметры:
    df1 - DataFrame с пользовательскими данными
    df2 - DataFrame с действиями пользователей (должен содержать 'target')
    
    Возвращает:
    Обученный pipeline
    """
    print("\n=== Начало обучения модели ===")
    
    # Сброс индексов для избежания проблем при объединении
    df1 = df1.reset_index(drop=True)
    df2 = df2.reset_index(drop=True)
    print("Индексы сброшены в обоих DataFrame")
    
    # Проверяем наличие target в df2
    if 'target' not in df2.columns:
        error_msg = "df2 должен содержать столбец 'target' для обучения"
        print(f"Ошибка: {error_msg}")
        raise ValueError(error_msg)
    
    print("\n=== Статистика входных данных ===")
    print(f"Размер df1: {df1.shape} (строки, колонки)")
    print(f"Колонки df1: {df1.columns.tolist()}")
    print(f"\nРазмер df2: {df2.shape} (строки, колонки)")
    print(f"Колонки df2: {df2.columns.tolist()}")
    print(f"\nРаспределение target в df2:\n{df2['target'].value_counts(normalize=True)}")
    
    # Создаем pipeline
    print("\n=== Создание pipeline ===")
    pipeline = build_pipeline(df1, df2)
    
    # Выделяем target из df2
    merged = pd.merge(df1, df2, on='session_id', how='left')
    merged['target'] = merged['target'].fillna(0)
    y = merged['target']
    print(f"\nЦелевая переменная y: {len(y)} значений")
    print(f"Пример значений y: {y[:5].tolist()}")
    
    # Обучаем pipeline
    print("\n=== Начало обучения pipeline ===")
    print("Важно: передаем df1, а target берется из объединенных данных в DataPreparing")
    start_time = time.time()
    
    pipeline.fit(df1, y)
    
    training_time = time.time() - start_time
    print(f"\nОбучение завершено за {training_time:.2f} секунд")
    
    print("\n=== Обученная модель готова ===")
    return pipeline

In [16]:
# Запуск процесса

import time
pipeline = train_model(train_sessions_df,train_hits_df)



=== Начало обучения модели ===
Индексы сброшены в обоих DataFrame

=== Статистика входных данных ===
Размер df1: (1488033, 18) (строки, колонки)
Колонки df1: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city']

Размер df2: (12539129, 12) (строки, колонки)
Колонки df2: ['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_category', 'event_action', 'event_label', 'event_value', 'target']

Распределение target в df2:
target
0    0.908905
1    0.091095
Name: proportion, dtype: float64

=== Создание pipeline ===

=== Начало построения пайплайна ===

=== DataPreparing.transform() ===
Входные данные (df1):
- Индексы: [0, 1, 2, 3, 4]... (всего: 1488033)
- Колонки: ['session_id', 'client_id', 'visit_date', 'visit_t

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value,target
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,3665.0,3.0,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN,0.0
1,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,46592.0,4.0,event,NaN,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,NaN,0.0



=== Начало transform ===
Исходный датафрейм: 12641170 строк, 29 колонок
Индексы: RangeIndex(start=0, stop=12641170, step=1)
Колонки: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_category', 'event_action', 'event_label', 'event_value', 'target']

После добавления временных признаков:
Размер: 12641170 строк, 32 колонок
Новые колонки: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path

In [17]:
# Функция для рассчета вероятности
from sklearn.base import clone  # Добавляем импорт clone
def predict_with_pipeline(pipeline, new_df1, new_df2):
    # Приводим индексы к одинаковым
    new_df1 = new_df1.reset_index(drop=True)
    new_df2 = new_df2.reset_index(drop=True)
    
    predict_pipe = clone(pipeline)
    if 'undersampler' in pipeline.named_steps:
        del pipeline.named_steps['undersampler']
    
    pipeline.named_steps['data_engineer'].df2 = new_df2
    y_proba = pipeline.predict_proba(new_df1)[:, 1]
    y_pred = pipeline.predict(new_df1)
    return y_proba,y_pred

In [18]:
# Готовим y_test для оценки качества
test_sessions_df = test_sessions_df  # DataFrame с сессиями (X)
test_hits_df = test_hits_df      # DataFrame с хитами (df2)

# 1. Объединяем с left join (как в DataPreparing)
merged = pd.merge(
    test_sessions_df, 
    test_hits_df, 
    on='session_id', 
    how='left'  # Важно: left join!
)

# 2. Заменяем специальные значения на NaN (как в DataPreparing)
merged = merged.replace(['(not set)', '(none)'], np.nan)

# 3. Проверяем результат (должно совпадать с DataPreparing)
print("Размер после объединения:", merged.shape)  # Должно быть (3171825, 29)


Размер после объединения: (3171825, 29)


In [19]:
# Готовим y_test для оценки качества и запускаем рассчет вероятности на тестовых данных(без столбца target)
merged['target'] = merged['target'].fillna(0)
y_test = merged['target']
test_hits_df = test_hits_df.drop(columns = ['target'])
y_proba,у_pred = predict_with_pipeline(pipeline, test_sessions_df,test_hits_df)


=== DataPreparing.transform() ===
Входные данные (X):
- Индексы: [0, 1, 2, 3, 4]... (всего: 372009)
- Колонки: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city']
- Размер: (372009, 18)

Данные df2:
- Индексы: [0, 1, 2, 3, 4]... (всего: 3146090)
- Колонки: ['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_category', 'event_action', 'event_label', 'event_value']
- Размер: (3146090, 11)

Результат объединения:
- Индексы: [0, 1, 2, 3, 4]... (всего: 3171825)
- Колонки: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'd

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-03,101.0,3.0,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
1,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-03,164184.0,10.0,event,NaN,sberauto.com/cars?city=1&rental_page=rental_on...,listing_ads,go_to_car_card,hAHqGICPFQiPwtzubOzs,NaN



=== Начало transform ===
Исходный датафрейм: 3171825 строк, 28 колонок
Индексы: RangeIndex(start=0, stop=3171825, step=1)
Колонки: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_category', 'event_action', 'event_label', 'event_value']

После добавления временных признаков:
Размер: 3171825 строк, 31 колонок
Новые колонки: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_cat

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-03,101.0,3.0,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
1,9055462349345527315.1638536723.1638536723,2108389127.1638536723,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-03,164184.0,10.0,event,NaN,sberauto.com/cars?city=1&rental_page=rental_on...,listing_ads,go_to_car_card,hAHqGICPFQiPwtzubOzs,NaN



=== Начало transform ===
Исходный датафрейм: 3171825 строк, 28 колонок
Индексы: RangeIndex(start=0, stop=3171825, step=1)
Колонки: ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_category', 'event_action', 'event_label', 'event_value']

После добавления временных признаков:
Размер: 3171825 строк, 31 колонок
Новые колонки: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'hit_date', 'hit_time', 'hit_number', 'hit_type', 'hit_referer', 'hit_page_path', 'event_cat

In [20]:
# Оцениваем качество
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import auc, precision_recall_curve
print(f"Test ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}")
precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)
print(f"PR-AUC: {pr_auc:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, у_pred))
print("\nConfusion Matrix:") 
print(confusion_matrix(y_test, у_pred))

Test ROC-AUC: 0.8773
PR-AUC: 0.575

Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.80      0.88   2887485
         1.0       0.28      0.78      0.41    284340

    accuracy                           0.80   3171825
   macro avg       0.62      0.79      0.64   3171825
weighted avg       0.91      0.80      0.83   3171825


Confusion Matrix:
[[2302529  584956]
 [  62368  221972]]
